In [12]:
import cassandra

In [13]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [14]:
try:
    session.execute("""select * from manga_libary""")
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="No keyspace has been specified. USE a keyspace, or explicitly specify keyspace.tablename"


In [15]:
try:
    session.execute("""
    create keyspace if not exists miketest
    with replication=
    {'class':'SimpleStrategy','replication_factor':1
    }"""
    )
except Exception as e:
    print(e)

In [16]:
try:
    session.set_keyspace('miketest')
except Exception as e:
    print(e)

In [21]:
query = "CREATE TABLE IF NOT EXISTS manga_library "
query = query + "(year int, author_name text, manga_name text, finish boolean, PRIMARY KEY (year, author_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [22]:
query = "select count(*) from manga_library"
try:
    count = session.execute(query)
except Exception as e:
    print(e)
    
print(count.one())

Row(count=0)


In [23]:
query = "INSERT INTO manga_library (year, author_name, manga_name, finish)"
query = query + " VALUES (%s, %s, %s, %s)"

try:
    session.execute(query, (1990, "Takehiko Inoue", "Slam Dunk", True))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1997, "Eiichiro Oda", "One Piece", False))
except Exception as e:
    print(e)

In [24]:
query = 'SELECT * FROM manga_library'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.year, row.manga_name, row.author_name, row.finish)

1990 Slam Dunk Takehiko Inoue True
1997 One Piece Eiichiro Oda False


In [26]:
### not use partitio key with query condition.

In [29]:
query = "select * from manga_library WHERE  author_name='Takehiko Inoue'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.year, row.manga_name, row.author_name)

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


In [ ]:
### not use clustering column with query condition

In [30]:
query = "select * from manga_library WHERE  year = 1990 and manga_name='Slam Dunk'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.year, row.manga_name, row.author_name)

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


In [ ]:
### use partition key and clustering column with query condition

In [31]:
query = "select * from manga_library WHERE  year = 1990 and author_name='Takehiko Inoue'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.year, row.manga_name, row.author_name)

1990 Slam Dunk Takehiko Inoue


In [32]:
query = "drop table manga_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [33]:
session.shutdown()
cluster.shutdown()